# We want to convert text -> numerical values

The Goals are:
* Create Vocabulary mapping each word to an index
* We need to setup PyTorch dataset to load the data
* Setup padding of every padding (all example should be of the same sequence length)

In [1]:
import re
import pathlib
import numpy as np
from pathlib import Path
from MSCOCO import COCO
from Vocabulary import Vocabulary
from collections import deque
from itertools import islice

from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

In [2]:
import nltk
nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/alexxcollins/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     /Users/alexxcollins/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [3]:
root = Path('./Datasets/coco')
imgs_path = root/'images'/'train2017'
captions_path = root/'annotations'/'captions_train2017.json'

In [4]:
coco = COCO(imgs_path, captions_path)

vocab_class = Vocabulary(4, 6)
vocab_class2 = Vocabulary(4, 6)

sentence_list = coco.captions_to_list()

Test the original `build_vocabulary` method:

In [5]:
%%time
vocab_class.build_vocabulary(sentence_list)

CPU times: user 47 s, sys: 58 ms, total: 47 s
Wall time: 47.3 s


In [6]:
len(vocab_class.idx_to_string), len(vocab_class.string_to_index)

(11221, 11221)

Test the new `build_vocaulary` method

In [7]:
%%time
vocab_class2.build_vocab2(sentence_list)

CPU times: user 22 s, sys: 394 ms, total: 22.4 s
Wall time: 22.6 s


In [8]:
len(vocab_class.idx_to_string), len(vocab_class.string_to_index)

(11221, 11221)

In [9]:
v1 = sorted([key for key in vocab_class.string_to_index])
v2 = sorted([key for key in vocab_class2.string_to_index])
v1 == v2

True

In [10]:
v1[:10]

['0', '00', '1', '10', '100', '10th', '11', '11th', '12', '13']

In [11]:
v2[:10]

['0', '00', '1', '10', '100', '10th', '11', '11th', '12', '13']